# Mandelbrot Set

## Pillow

Let's import the Pillow library and abstract away the drawing logic: 

In [1]:
from typing import NamedTuple
from math import log, log2

import PIL.Image
from PIL.ImageColor import getrgb


class Viewport(NamedTuple):
    image: PIL.Image.Image
    xmin: float
    xmax: float

    @property
    def width(self):
        return self.xmax - self.xmin

    @property
    def height(self):
        return self.width * self.aspect_ratio

    @property
    def aspect_ratio(self):
        return self.image.height / self.image.width

    @property
    def scale(self):
        return self.width / self.image.width

    def __iter__(self):
        for x in range(self.image.width):
            for y in range(self.image.height):
                yield Pixel(self, x, y)


class Pixel(NamedTuple):
    viewport: Viewport
    x: int
    y: int

    @property
    def image(self):
        return self.viewport.image

    @property
    def color(self):
        return self.image.getpixel((self.x, self.y))

    @color.setter
    def color(self, value):
        return self.image.putpixel((self.x, self.y), value)

    def __complex__(self):
        return complex(
            self.viewport.scale * self.x + self.viewport.xmin,
            self.viewport.scale * (self.image.height / 2 - self.y),
        )


def hsb(hue_degrees: int, saturation: float, brightness: float):
    """Return a tuple of RGB values in 0..255 range."""
    return getrgb(
        f"hsv({hue_degrees % 360}, {saturation * 100}%, {brightness * 100}%)"
    )

## Mandelbrot

Here's a class that models the Mandelbrot set:

In [2]:
class Mandelbrot(NamedTuple):
    """The Mandelbrot set.

    >>> mandelbrot_set = Mandelbrot(max_iterations=100)
    >>> 0.5 + 0.2j in mandelbrot_set
    False
    >>> mandelbrot_set.escape_count(0.5 + 0.2j)
    4
    >>> mandelbrot_set.probability(0.5 + 0.2j)
    0.04
    >>> mandelbrot_set.probability(0.5 + 0.2j, smooth=True)
    0.05147130599503858
    """

    max_iterations: int

    def __contains__(self, c: complex) -> bool:
        """Return True if a number belongs to the set."""
        return self.probability(c) == 1

    def probability(self, c: complex, smooth: bool = False) -> float:
        """Return a fuzzy degree of membership."""
        return self.escape_count(c, smooth) / self.max_iterations

    def escape_count(self, c: complex, smooth: bool = False) -> int:
        """Return the number of iterations before bailing out."""
        z = 0j
        for i in range(self.max_iterations):
            z = z ** 2 + c
            if abs(z) >= 2:
                return i + 1 - log(log(abs(z))) / log(2) if smooth else i
        return self.max_iterations

## Drawing the Mandelbrot Set

In [3]:
from ipywidgets import interact, IntSlider, FloatSlider
from IPython.display import display

width_px = height_px = 150

@interact(
    mode=["Black & White", "Escape Count", "Escape Count (Smooth)", "Colored (Smooth)"],
    max_iterations=IntSlider(min=1, max=25, value=20),
    xmin=FloatSlider(min=-5, max=0, value=-1.75),
    xmax=FloatSlider(min=0, max=5, value=0.5)
)
def draw(mode, max_iterations, xmin, xmax):
    
    mandelbrot_set = Mandelbrot(max_iterations)
    
    if mode == "Black & White":
        image = PIL.Image.new("L", (width_px, height_px), "#fff")
        for pixel in Viewport(image, xmin, xmax):
            if complex(pixel) in mandelbrot_set:
                pixel.color = 0
    elif mode == "Escape Count":
        image = PIL.Image.new("L", (width_px, height_px))
        for pixel in Viewport(image, xmin=-2.5, xmax=1.5):
            probability = mandelbrot_set.probability(complex(pixel))
            pixel.color = 255 - int(255 * probability)
    elif mode == "Escape Count (Smooth)":
        image = PIL.Image.new("L", (width_px, height_px))
        for pixel in Viewport(image, xmin=-2.5, xmax=1.5):
            probability = mandelbrot_set.probability(complex(pixel), smooth=True)
            pixel.color = 255 - int(255 * probability)
    elif mode == "Colored (Smooth)":
        image = PIL.Image.new("RGB", (width_px, height_px))
        for pixel in Viewport(image, xmin=-2.5, xmax=1.5):
            probability = mandelbrot_set.probability(complex(pixel), smooth=True)
            pixel.color = hsb(
                hue_degrees=int(probability * 360),
                saturation=probability,
                brightness=1
            )
    
    display(image)

interactive(children=(Dropdown(description='mode', options=('Black & White', 'Escape Count', 'Escape Count (Sm…